In [1]:
import pandas as pd
import math
from dateutil.relativedelta import relativedelta
from datetime import datetime
import re
import copy
from sqlalchemy import create_engine

In [2]:
conn_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(user='postgres',
                                                                     pwd='123',
                                                                     host='localhost',
                                                                     port='5432',
                                                                     db='postgres'
                                                                     )
engine = create_engine(conn_string, paramstyle='format', isolation_level="READ UNCOMMITTED")

In [ ]:
conn_string_cloud = conn_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(user='postgres',
                                                                     pwd='',
                                                                     host='10.247.204.254',
                                                                     port='5433',
                                                                     db='test_vow4'
                                                                     )
engine2 = create_engine(conn_string_cloud, paramstyle='format', isolation_level="READ UNCOMMITTED")
con_test_qry = pd.read_sql_query('Select * from vm_vh_class',con=engine2)
if con_test_qry.shape == (70,5):
    print("Connection Established")
else:
    print("Error in Connection")

In [3]:
path = 'D:/RJ/'
state = 'RJ'

In [4]:
slab_fields = pd.read_excel(r'C:\Users\CDMA\Downloads\slab_fields_prep_v2.xlsx',sheet_name = 'slab_fields_prep')

In [5]:
slab_new = pd.read_csv(r'C:\Users\CDMA\Downloads\vm_tax_slab_new_transformed.csv')
slab_new = slab_new[slab_new['state_cd']==state]
slab_new = slab_new[slab_new['vh_class'].notnull()]

In [6]:
slab_new['date_from'] = slab_new['date_from'].astype('datetime64[ns]')
slab_new['date_to'] = slab_new['date_to'].astype('datetime64[ns]')

In [7]:
slab_add_new = pd.read_csv(r'D:\Jharkhand python\vm_tax_slab_add_new.csv')
slab_add_new = slab_add_new[slab_add_new['state_cd']==state]

In [8]:
vt_tax_based_on = pd.read_csv(path+'/vt_tax_based_on.csv')
#vt_tax_based_on_permit_info = pd.read_csv(r'D:\AS\vt_tax_based_on_permit_info.csv')
#vt_tax_based_on = pd.read_sql_query('Select * from vt_tax_based_on where state_cd ='+"'{}'".format(state),con=engine2)

In [9]:
vm_vh_class=pd.read_csv(r'D:\Jharkhand python\vm_vh_class.csv')


In [10]:
vt_owner = pd.read_csv(path+'/vt_owner.csv')
#vt_owner = pd.read_sql_query('Select * from vt_owner where state_cd ='+"'{}'".format(state),con=engine2)

C:\Users\CDMA\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (52) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [11]:
vt_permit = pd.read_csv(path+'/vt_permit.csv')
#vt_owner = pd.read_sql_query('Select * from vt_owner where state_cd ='+"'{}'".format(state),con=engine2)

In [12]:
tm_office = pd.read_csv(path+'/tm_office.csv')
tm_office = tm_office[tm_office['state_cd']==state]

In [82]:
vt_tax = pd.read_csv(path+'vt_tax.csv')
#vt_tax = pd.read_sql_query('Select * from vt_tax where state_cd ='+"'{}'".format(state),con=engine2)
vt_tax = vt_tax[vt_tax['Pur Cd']==58]
vt_tax = vt_tax[vt_tax['Tax Mode']!='B']
#vt_tax=vt_tax[vt_tax['Collected By']!='VAHAN2']
vt_tax.reset_index(drop=True, inplace=True)


C:\Users\CDMA\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [13]:
#Turn into first letter capital in each word
def change_col_names(cols):
    pp=[]
    for j in cols :
        p= j.replace('_',' ').title()
        pp.append(p)
        #df.rename(columns={j: p},inplace=True)
    return pp

In [13]:
def_list = pd.read_excel(r'C:\Users\CDMA\Downloads\Tax Defaulter Report jagatpura jaipur.xls')
def_list = def_list[def_list['Tax Arrear From'].notnull()]
def_list.columns

Index(['S. No', 'Select Demand Notice', 'Regn No', 'Office', 'Regn Date',
       'Vehicle Class', 'Vehicle Category', 'purpose Code', 'Tax Type',
       'Owner Name', 'Father Name', 'Complete Present Address', 'Mobile No',
       'Laden Wt.', 'UnLaden Wt.', 'Seating Capacity', 'Standing Capacity',
       'Fuel Type', 'Tax From', 'Tax Upto', 'Tax Cleared Upto',
       'Tax Arrear From', 'Tax Arrear UpTo', 'Permit Valid From',
       'Permit Valid Upto', 'Tax Amount in (Rs)', 'Tax Fine in (Rs)',
       'Total Tax Amount in (Rs)', 'Fitness Validity', 'Financer Name',
       'Blacklist Status'],
      dtype='object')

In [14]:
def_list['Vehicle Class'].unique()

array(['Three Wheeler (Passenger)', 'Three Wheeler (Goods)',
       'Goods Carrier', 'Articulated Vehicle', 'Dumper', 'Bus',
       'Educational Institution Bus', 'Omni Bus'], dtype=object)

In [15]:
def_list = def_list[def_list['Vehicle Class']=='Goods Carrier']

In [67]:
def calculate_formula(formula_df,params_info):
    #Calculate the Formula one by one
    ind=[]
    cff_formula = []
    cc_formula=[]
    ccf_result=[]
    cr_rate=[]
    #c_add_code=[]
    #reb_df = formula_df[formula_df['type_flag']=='R']
    for l in range(len(formula_df)):
        #print('l',l)
        condition_formula = formula_df['condition_formula'].to_list()[l]
        if '\r' in condition_formula:
            condition_formula = condition_formula.replace('\r','')
        if '\n' in condition_formula:
            condition_formula = condition_formula.replace('\n','')

        cf_params = []
        for k in re.split(r'[()\*\s]\s*',condition_formula):
            if '<' in k and '>' in k:
                cf_params.append(k[k.find('<'):(k.find('>')+1)])
        cf_params = list(set(cf_params))
        c_formula = condition_formula
        c_formula = condition_form_convert(cf_params,params_info, c_formula)
        #print(c_formula)
        for m in cf_params:
            if m != '<>':
                val = str(params_info[params_info['code']==m]['P_value'].to_list()[0])
                c_formula = c_formula.replace(str(m),val)
        
        if c_formula.lower() != 'false' and formula_df['type_flag'].to_list()[l] != 'R' and 'nan' not in c_formula.lower():
            cf_result = pd.read_sql_query(repr('select '+c_formula)[1:-1],con=engine)
            cf_result = cf_result.loc[0,'?column?']
        else:
            cf_result = False
        if cf_result==True:
            #print('Ind',ind,l)
            ind.append(l)
        cff_formula.append(condition_formula)
        cc_formula.append(c_formula)
        ccf_result.append(cf_result)
        cr_rate.append(formula_df.loc[:,'rate_formula'].to_list()[l])
        
    formula_df.reset_index(drop=True, inplace=True)
    summary_df = pd.DataFrame(list(zip(cff_formula,cc_formula, ccf_result,cr_rate)), columns =['Condition Formula', 'Condition Result','Result','Rate Formula']) 
    return [ind,summary_df]

In [17]:
def condition_form_convert(cf_params,params_info, c_formula):
    for m in cf_params:
        if m != '<>':
            val = str(params_info[params_info['code']==m]['P_value'].to_list()[0])
            # Change '' to \'\' in case of string val
            if val.isdigit()==False:
                if '.' in val:
                    v = val.replace('.','')
                    if v.isdigit():
                        val=val
                    else:
                        val=repr(val)
                else:
                    val=repr(val)
            else:
                val=val
            c_formula = c_formula.replace(str(m),val)
    return c_formula
'''
        else:
            c_formula = c_formula.replace('<>','!=')

    # Change '=' to '=='
    if '<=' in c_formula:
        c_formula=c_formula.replace('<=','@')
    if '>=' in c_formula:
        c_formula=c_formula.replace('>=',';')
    if '!=' in c_formula:
        c_formula=c_formula.replace('!=','?')
    if '=' in c_formula:
        c_formula = c_formula.replace('=','==')
    if '@' in c_formula:
        c_formula=c_formula.replace('@','<=')
    if ';' in c_formula:
        c_formula=c_formula.replace(';','>=')
    if '?' in c_formula:
        c_formula = c_formula.replace('?','!=')
    return c_formula
'''

"\n        else:\n            c_formula = c_formula.replace('<>','!=')\n\n    # Change '=' to '=='\n    if '<=' in c_formula:\n        c_formula=c_formula.replace('<=','@')\n    if '>=' in c_formula:\n        c_formula=c_formula.replace('>=',';')\n    if '!=' in c_formula:\n        c_formula=c_formula.replace('!=','?')\n    if '=' in c_formula:\n        c_formula = c_formula.replace('=','==')\n    if '@' in c_formula:\n        c_formula=c_formula.replace('@','<=')\n    if ';' in c_formula:\n        c_formula=c_formula.replace(';','>=')\n    if '?' in c_formula:\n        c_formula = c_formula.replace('?','!=')\n    return c_formula\n"

In [72]:
def calculate_rates(true_df,params_info):
    t_params = []
    for q in range(len(true_df)):
        r_formula = true_df['rate_formula'].to_list()[q]
        for k in re.split(r'[()\*\s]\s*',r_formula):
            if '<' in k and '>' in k:
                t_params.append(k[k.find('<'):(k.find('>')+1)])

    t_params = list(set(t_params))
    #print('t_params',t_params)
    rates=[]
    for q in range(len(true_df)):
        r_formula = true_df['rate_formula'].to_list()[q]
        for m in t_params:
            r_formula = r_formula.replace(str(m),str(params_info[params_info['code']==m]['P_value'].to_list()[0]))
        
        #print('r_formula',r_formula)
            
        #rate = eval(r_formula.lower())
        rate = pd.read_sql_query(repr('select '+r_formula)[1:-1],con=engine)
        rate = rate.loc[0,'?column?']
        #print(rate)
        rates.append(rate)        
    true_df['Rate']=rates
    return true_df

In [19]:
def tbo_last_rcpt(regn_no,before):
    tbo_df = vt_tax_based_on.loc[vt_tax_based_on['regn_no']==regn_no]
    tbo_df= copy.deepcopy(tbo_df)
    #print(tbo_df.shape)
    #tbo_df = tbo_df.reset_index(drop=True, inplace=True)
    tbo_df['op_dt'] = tbo_df.loc[:,'op_dt'].apply(lambda x: datetime.strptime(x[:10], '%d-%m-%Y'))
    tbo_df = tbo_df.sort_values(by='op_dt', ascending=False)
    #tbo_df = tbo_df.reset_index(drop=True, inplace=True)
    if len(tbo_df)>0:#print(tbo_df.shape)
        for n in tbo_df.index:
            x = before-tbo_df.loc[n,'op_dt']
            #x = x.years*365 + x.months*30 + x.days
            if x.days>=0:
                last_rcpt = tbo_df.loc[n,'rcpt_no']
                break
            last_rcpt=''
        if last_rcpt=='':
            print('No prior receipt info!!! Rcpt:',' Regn No:',regn_no)
            return 0
        else:
            return last_rcpt
    else:
        return 0

In [20]:
def owner_or_tbo(regn_no,params_info,param_info_owner,ot_df,tab,new_vch,rcpt_dt,tax_mode):
    for pi in range(len(param_info_owner)):
        col = param_info_owner['column_name'].to_list()[pi][1:-2]
        par = param_info_owner['column - slab_field'].to_list()[pi]
        code = param_info_owner['code'].to_list()[pi]
        #col= col.replace('_',' ').title()
        p_value = ''
        #print(col,par,code,tab)
        purchase_dt = ot_df['purchase_dt'].to_list()[0]
        if purchase_dt[:4].isdigit():
            purchase_dt =datetime.strptime(ot_df['purchase_dt'].to_list()[0],'%Y-%m-%d')
        else:
            purchase_dt =datetime.strptime(ot_df['purchase_dt'].to_list()[0],'%d-%m-%Y')
        if par=='vch_age':
            p_value = relativedelta(rcpt_dt, purchase_dt)
            p_value = p_value.years
        elif par=='vch_age_months':
            p_value = relativedelta(rcpt_dt, purchase_dt)
            p_value = (p_value.years *12) + p_value.months
        elif par=='purchase_dt_vch_age':
            p_value = relativedelta(rcpt_dt, purchase_dt)
            p_value = p_value.years
        elif par=='fin_vch_age':
            #print(datetime.strptime(ot_df['purchase_dt'].to_list()[0],'%d-%m-%Y'))
            p_value = relativedelta(rcpt_dt, purchase_dt)
            p_value = p_value.years
        elif par=='tax_month_duration':
            p_value = relativedelta(tax_upto, tax_from)
            p_value = p_value.years *12 + p_value.months
        elif par=='payment_date':
            p_value = rcpt_dt
        #elif par=='distance_run_in_month':
        #    p_value = vt_tax_based_on['distance_run_in_quarter']/4
        #elif par=='transaction_pur_cd':
        #    appl_no = vp_appl_rcpt_mapping[vp_appl_rcpt_mapping['rcpt_no']==rcpt_no]['appl_no'].to_list()[0]
        #    if len(va_details[va_details['appl_no']==appl_no][col].to_list())==0:
        #        print(i,'No pur_cd available in va_details')
        #        print('Appl no',appl_no,' !!! Rcpt:',rcpt_no,' Regn No:',regn_no,'vh_class',vh_class)
        #        continue
        #    else:
        #        p_value = va_details[va_details['appl_no']==appl_no][col].to_list()[0]
        #print(p_value)
        elif par=='vch_type':
            p_value = vm_vh_class[vm_vh_class['vh_class']==vh_class]['Class Type'].to_list()[0]
        elif par=='new_vch':
            p_value = new_vch
        elif par=='tax_mode':
            p_value = tax_mode
        elif par=='permit_sub_catg':
            if 'pmt_catg' in ot_df.columns:
                p_value = ot_df['pmt_catg'].to_list()[0]
            else:
                p_value = float("nan")
        elif par=='delay_days':
            #print(rcpt_dt[:10],' ',tax_from)
            p_value = rcpt_dt-tax_from
            #p_value = p_value.years * 365 + p_value.months * 30 + p_value.days
            if p_value.days > 0:
                p_value = p_value.days
            elif p_value == 0:
                p_value = 1
            else:
                p_value = 0
            #print(p_value)
        else:
            #if tab=='own':
                #col= col.replace('_',' ').title()
            p_value= ot_df[col].to_list()[0]
            #print(p_value)
        #print(col,' ',par,' ',code,' ',p_value)
        params_info.loc[params_info.index[params_info['code'] == code].tolist()[0],'P_value']=p_value
        #print(code,p_value)
    return params_info

In [21]:
def extract_parameters(regn_no,tax_from,rcpt_dt,params_info,param_info_tbo,param_info_owner,param_info_other,new_vch,pur_cd,tax_mode,final_rate):
    if len(param_info_owner)>0:
        owner_df = vt_owner.loc[vt_owner['regn_no']==regn_no]
        owner_df= copy.deepcopy(owner_df)
    if len(param_info_tbo)>0:
        last_rcpt = tbo_last_rcpt(regn_no,rcpt_dt)
        if last_rcpt !=0:
            tbo_df = vt_tax_based_on.loc[vt_tax_based_on['rcpt_no']==last_rcpt]
            c = tbo_df.columns
            #for j in c :
                #p= j.replace('_',' ').title()
                #tbo_df.rename(columns={j: p},inplace=True)
            tbo_df= copy.deepcopy(tbo_df)
        else:
            tbo_df=[]

    # Extract parameters from the TBO table
    if len(tbo_df)>0 :
        #print('TBO')
        tab = 'tbo'
        #print(tbo_df.columns)
        params_info = owner_or_tbo(regn_no,params_info,param_info_tbo,tbo_df,tab,new_vch,rcpt_dt,tax_mode)
        #print('TBO',params_info['P_value'].to_list())
    else:
        #print('Owner')
        tab='own'
        params_info = owner_or_tbo(regn_no,params_info,param_info_owner,owner_df,tab,new_vch,rcpt_dt,tax_mode)
    #print('Shape',params_info)
    if len(param_info_other)>0:  
        #print('Other')
        #print(param_info_other.shape)
        for pi in range(len(param_info_other)):
            col = param_info_other['column_name'].to_list()[pi][1:-2]
            par = param_info_other['column - slab_field'].to_list()[pi]
            code = param_info_other['code'].to_list()[pi]
            
            #print(null_df)
            #print('Yes!',)
            #print('Other',code,col,par)
            #if math.isnan(params_info[params_info['code']==code]['P_value'].to_list()[0]):
            if pd.isnull(params_info.loc[params_info['code']==code]['P_value']).to_list()[0]==True:
                if col == 'domain_cd':
                    p_value = vt_permit[vt_permit['Regn No']==regn_no]['Domain Cd'].to_list()
                    if len(p_value)!=0:
                        p_value = p_value[0]
                    #elif len(vt_tax_based_on[vt_tax_based_on['rcpt_no']==tbo_last_rcpt(regn_no,rcpt_dt)]['domain_cd'].to_list())>0:
                    #    p_value = vt_tax_based_on[vt_tax_based_on['rcpt_no']==tbo_last_rcpt(regn_no,rcpt_dt)]['domain_cd'].to_list()[0]
                    else:
                        p_value = None
                        print('No Domain Cd Available')
                elif col == 'pmt_type':
                    p_value = vt_permit[vt_permit['Regn No']==regn_no]['Pmt Type'].to_list()
                    if len(p_value)!=0:
                        p_value = p_value[0]
                    #elif len(vt_tax_based_on[vt_tax_based_on['rcpt_no']==tbo_last_rcpt(regn_no,rcpt_dt)]['pmt_type'].to_list())>0:
                    #    p_value = vt_tax_based_on[vt_tax_based_on['rcpt_no']==tbo_last_rcpt(regn_no,rcpt_dt)]['pmt_type'].to_list()[0]
                    else:
                        p_value = None
                        print('No Permit Type Available')
                elif col== 'pur_cd':
                    if pur_cd==57 or pur_cd == 58:
                        p_value=58
                elif col=='tax_amt':
                    p_value = final_rate
                #print('Other',code,col,par)
                params_info.loc[params_info.index[params_info['code'] == code].tolist()[0],'P_value']=p_value  
    return params_info

In [79]:
date_format = "%d-%b-%Y"
def calculate_tax(regn_no,tax_from,tax_upto='2019-10-03', pur_cd = 58,final_rate=0,tax_mode='Q' ,state_cd='RJ',off_cd=141):
    rcpt_dt = tax_upto
    
    if len(vt_owner[vt_owner['regn_no']==regn_no])>0:
        vh_class = vt_owner[vt_owner['regn_no']==regn_no]['vh_class'].to_list()[0]
        new_vch = 'N'
        #print(vh_class)
    else:
        print(i,'NO INFO IN VT_OWNER',regn_no)
        return 0
        
    #slab_codes = slab_new[(slab_new['vh_class']==vh_class) & (slab_new['pur_cd']==pur_cd) & (slab_new['tax_mode']==tax_mode) & (slab_new['state_cd']==state_cd) & (slab_new['date_from']<tax_from) & (slab_new['date_to']>tax_from) ]['slab_code'].to_list()
    #tax_from = datetime.strptime(tax_from,'%d-%b-%Y')
    #print(vh_class, pur_cd, tax_mode, tax_from)
    slab_new1 = slab_new[(slab_new['vh_class']==vh_class) & (slab_new['pur_cd']==pur_cd) & (slab_new['tax_mode']==tax_mode) &(slab_new['state_cd']==state_cd) & (slab_new['date_from'] < tax_from) & (slab_new['date_to'] > tax_from)]
    #print(slab_new1)
    slab_codes = slab_new1['slab_code'].to_list()
    #print(pur_cd,'Slab Codes Applicable:',slab_codes)
    if len(slab_codes)==0:
        print(i,'Missed',regn_no,'vh_class',vh_class)
        return 0

    formula_df = slab_add_new.loc[(slab_add_new['slab_code'].isin(slab_codes)) & (slab_add_new['state_cd']==state_cd)]
    formula_df = formula_df[formula_df['condition_formula']!='0=0']
    #formula_df = formula_df[formula_df['condition_formula']!='1=1']
    #formula_df.columns
    
    #print(formula_df)
        
    #Identify the parameters required for calculating the formulas
    params = []
    for j in range(len(formula_df)):
        formula = formula_df['condition_formula'].to_list()[j] +' '+formula_df['rate_formula'].to_list()[j]
        for k in re.split(r'[()\*\s]\s*',formula):
            if '<' in k and '>' in k:
                params.append(k[k.find('<'):(k.find('>')+1)])
    params = list(set(params))
    #print('Required Parameters :',params)
    params_info = slab_fields.loc[slab_fields['code'].isin(params)]
    params_info = params_info.reindex(columns = params_info.columns.tolist() + ['P_value'])
    #print(params_info)
    param_info_owner = params_info[params_info['def_table']=='vt_owner']
    #print('Owner',param_info_owner['column - slab_field'].to_list())
    param_info_other = params_info[params_info['def_table']!='vt_owner']
    #print('Other',param_info_other['column - slab_field'].to_list())
    #param_info_tbopi = params_info[params_info['table - slab_field']=='vt_tax_based_on_permit_info']
    #param_info_misc = params_info[(params_info['table - slab_field']!='vt_tax_based_on_permit_info') & (params_info['table - slab_field']!='vt_tax_based_on')]
    param_info_tbo =  params_info[params_info['table - slab_field']=='vt_tax_based_on']
    #print('TBO',param_info_tbo['code'].to_list)
    #print('TBO',param_info_tbo['column - slab_field'].to_list())
    #print('Initial Shape',params_info.shape)
    params_info = extract_parameters(regn_no,tax_from,rcpt_dt,params_info,param_info_tbo,param_info_owner,param_info_other,new_vch,pur_cd,tax_mode,final_rate)
    #print('Final Shape',params_info.shape)                                                                        
    #if pur_cd ==57:        
    #print(params_info)
    ind = calculate_formula(formula_df,params_info)[0]
    summary_df = calculate_formula(formula_df,params_info)[1]
    print(summary_df)
    if len(ind)==0:
        print('No True Condition')
        return 
    #Module for calculation of rates for true conditions
    true_df = formula_df.loc[ind,:]
    true_df.reset_index(drop=True, inplace=True)
    #print('True_df',true_df)

    #print(t_params)
    true_df_t = true_df[(true_df['type_flag']=='T') | (true_df['type_flag']=='M') | (true_df['type_flag']=='X') ]
    
    true_df_r = true_df[true_df['type_flag']=='R']
    true_df_s = true_df[true_df['type_flag']=='S']
    
    true_df_t = calculate_rates(true_df_t,params_info)
    #print('T',true_df_t)
    #print('S',true_df_s)
    #print(rates)
    #print(true_df)
    #Module for dealing Minimum M and Amount T 
    if len(set(true_df_t['condition_formula'].to_list()))>0:
        #true_df_t = true_df_t[true_df_t['condition_formula']!='1=1']
        if len({'T','M'} - set(true_df_t['type_flag'].to_list()))==0:
            if true_df_t[true_df_t['type_flag']=='T']['Rate'].to_list()[0]> true_df_t[true_df_t['type_flag']=='M']['Rate'].to_list()[0]:
                final_rate = true_df_t[true_df_t['type_flag']=='T']['Rate'].to_list()
            else:
                final_rate = true_df_t[true_df_t['type_flag']=='M']['Rate'].to_list()
        elif len({'T','X'} - set(true_df_t['type_flag'].to_list()))==0:
            #print('Here')
            final_rate = true_df_t[true_df_t['type_flag']=='X']['Rate'].to_list()
        else:
            final_rate = true_df_t[true_df_t['type_flag']=='T']['Rate'].to_list()
    #else:       
    #    final_rate = true_df['Rate'].to_list()
    if type(final_rate)==list:
        final_rate = final_rate[0]
    #else:
    #    final_rate = 0
    #print(final_rate)
    
    #Module for Surcharge
    if len(true_df_s) > 0 :
        params_info.loc[params_info['code'] == '<58>','P_value']=final_rate
        true_df_s = calculate_rates(true_df_s,params_info)
        #print(true_df_s)
        surcharge = true_df_s[true_df_s['type_flag']=='S']['Rate'].to_list()
        #print(final_rate,surcharge)
        final_rate = final_rate + surcharge[0]
    '''
    if len(true_df_r) > 0:
        rr_formula = true_df_r[true_df_r['condition_formula']==f]['rate_formula'].to_list()[0]
        ra_formula = condition_form_convert(reb_params,params_info,rr_formula)
        reb_rate_result = eval(ra_formula.lower())              
        final_rate = final_rate - reb_rate_result
         
    #Module for Tax Mode Remaining years
    if tax_mode in ['Y15','Y10','Y12','Y20']:
        tax_period = relativedelta(datetime.strptime(tax_upto, date_format), datetime.strptime(tax_from, date_format))
        tax_period = tax_period.years*365 + tax_period.months*30 + tax_period.days
        #print('Tax Upto - Tax From =',tax_period,'days')
        veh_age = relativedelta(datetime.strptime(tax_from, date_format), datetime.strptime(vt_owner[vt_owner['Regn No']==regn_no]['Purchase Dt'].to_list()[0],'%Y-%m-%d'))
        #print(((veh_age.years * 365) + (veh_age.months * 30) + veh_age.days))
        if tax_mode=='Y10':
            days_rem = 3652 - ((veh_age.years * 365) + (veh_age.months * 30) + veh_age.days)
            final_rate = (final_rate/3652)*days_rem
        if tax_mode=='Y12':
            days_rem = 4383 - ((veh_age.years * 365) + (veh_age.months * 30) + veh_age.days)
            #print('Days remaining in ',tax_mode,'=',days_rem,'days')
            final_rate = (final_rate/4383)*days_rem
        if tax_mode=='Y15':
            days_rem = 5478 - ((veh_age.years * 365) + (veh_age.months * 30) + veh_age.days)
            final_rate = (final_rate/5478)*days_rem
    '''                            
    #Module for Quarterly Mode    
    if tax_mode== 'Q':
        #print(tax_upto,' ',tax_from)
        tax_period = relativedelta(tax_upto, tax_from)
        #print(tax_period)
        years_q = tax_period.years * 4 
        #tax_period = tax_period.years*12 + tax_period.months
        if tax_period.months <=3:
            months_q = 1
        elif tax_period.months <=6:
            months_q = 2
        elif tax_period.months <=9:
            months_q = 3
        else:
            months_q = 4
        n_quart = years_q + months_q
        #print(n_quart)
        final_rate = final_rate * n_quart
    elif tax_mode == 'Y':
        tax_period = relativedelta(tax_upto, tax_from)
        years_y = tax_period.years * 12 
        months_y = math.ceil(tax_period.months + years_y)
        if tax_period.days > 0:
            months_y = months_y+1
        #print((final_rate / 12),months_y)
        final_rate = (final_rate / 12) * months_y

    #Display Calculated Amount
    return final_rate

In [80]:
nc_regn_no_list = []
for i in range(4,5):   
    regn_no = def_list['Regn No'].to_list()[i]
    print(regn_no)
    #state_cd = 'AS'
    #off_cd = 1
    vh_class = def_list['Vehicle Class'].to_list()[i]
    #rcpt_no = def_list['Rcpt No'].to_list()[i]
    #print(regn_no,rcpt_no)
    #tax_amt = def_list['Tax Amt'].to_list()[i]
    #rcpt_dt = def_list['Rcpt Dt'].to_list()[i]
    
    #tax_from = def_list['latest_tax_paid_upto'] + 1
    tax_from = def_list['Tax Arrear From'].to_list()[i]
    #print(tax_from)
    tax_from = datetime.strptime(tax_from, date_format)
    
    tax_upto = def_list['Tax Arrear UpTo'].to_list()[i]
    tax_upto = datetime.strptime(tax_upto,'%d-%b-%Y')
    #tax_upto = datetime.strptime('04-May-2020','%d-%b-%Y')
    #print('Delayed',tax_upto-tax_from)
    c_tax_amt = round(calculate_tax(regn_no,tax_from,tax_upto,tax_mode='Y'))
    #print(c_tax_amt)
    if c_tax_amt == 0:
        print('Unable to calculate tax')
        nc_regn_no_list.append(regn_no)
    tax_amt = pd.to_numeric(def_list['Tax Amount in (Rs)'].str.replace(',',''),errors='coerce').to_list()[i]
    
    tax_fine = pd.to_numeric(def_list['Tax Fine in (Rs)'].str.replace(',',''),errors='coerce').to_list()[i]
    if c_tax_amt !=0:
        c_tax_fine = round(calculate_tax(regn_no,tax_from,tax_upto, pur_cd = 57, final_rate = c_tax_amt,tax_mode ='M'))
    else:
        c_tax_fine = 0
    #print(tax_fine)
    if tax_amt!=c_tax_amt: 
        print('Mismatch',i,regn_no, vh_class,str(tax_from)[:11],str(tax_upto)[:11],tax_amt,c_tax_amt,c_tax_fine,tax_fine)
    else:
        print(i, regn_no,vh_class,str(tax_from)[:11],str(tax_upto)[:11], tax_amt, c_tax_amt,c_tax_fine,tax_fine)
    #summary_df = c_tax_amt[1]
    

GJ06AX5483
                  Condition Formula  \
0        <5> >= 300000 AND <36> > 5   
1       <5> >= 300000 AND <36> <= 5   
2                     <5> <= 300000   
3   <5> >= 300001 AND <5> <= 600000   
4  <5> >= 600001 AND <5> <= 1000000   
5                    <5> >= 1000001   
6                     <5> <= 300000   

                               Condition Result  Result  \
0               3300000.0 >= 300000 AND 4.0 > 5   False   
1              3300000.0 >= 300000 AND 4.0 <= 5    True   
2                           3300000.0 <= 300000   False   
3   3300000.0 >= 300001 AND 3300000.0 <= 600000   False   
4  3300000.0 >= 600001 AND 3300000.0 <= 1000000   False   
5                          3300000.0 >= 1000001    True   
6                           3300000.0 <= 300000   False   

                                        Rate Formula  
0                                      <58> * 0.0625  
1                                      <58> * 0.0625  
2         (FLOOR((<5> + 999) / 1000) *

C:\Users\CDMA\anaconda3\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


                                   Condition Formula  \
0  (<57> BETWEEN (CASE WHEN <59> = 'M' THEN 8 ELS...   
1  (<57> BETWEEN (CASE WHEN <59> = 'M' THEN 30 EL...   
2  (<57> BETWEEN (CASE WHEN <59> = 'M' THEN 8 ELS...   
3  (<57> BETWEEN (CASE WHEN <59> = 'M' THEN 30 EL...   
4  (<57> BETWEEN (CASE WHEN <59> = 'M' THEN 8 ELS...   
5  <59>='Y' AND <60>='N' AND  <38> IN ('HGV','MGV...   

                                    Condition Result  Result  \
0  (730 BETWEEN (CASE WHEN 'M' = 'M' THEN 8 ELSE ...    True   
1  (730 BETWEEN (CASE WHEN 'M' = 'M' THEN 30 ELSE...   False   
2  (730 BETWEEN (CASE WHEN 'M' = 'M' THEN 8 ELSE ...   False   
3  (730 BETWEEN (CASE WHEN 'M' = 'M' THEN 30 ELSE...   False   
4  (730 BETWEEN (CASE WHEN 'M' = 'M' THEN 8 ELSE ...   False   
5  'M'='Y' AND 'N'='N' AND  'HGV' IN ('HGV','MGV'...   False   

                           Rate Formula  
0    (<58>*0.015) * (CEIL(<57> / 30.0))  
1  (<58>*0.015) * ((CEIL(<57> / 30.0)))  
2  (<58>*0.015) * ((CEIL(<57> / 

In [78]:
vt_owner[vt_owner['regn_no']=='GJ06AX5483']['sale_amt']

12876552    2020000
Name: sale_amt, dtype: int64

In [100]:
datetime.strptime(vt_owner[vt_owner['regn_no']=='GJ06AX5483']['op_dt'].to_list()[0][:10],'%Y-%m-%d') > datetime.strptime(vt_tax_based_on[vt_tax_based_on['regn_no']=='GJ06AX5483']['op_dt'].to_list()[0][:10],'%d-%m-%Y')

True

In [87]:
vt_tax_based_on[vt_tax_based_on['regn_no']=='GJ06AX5483']['op_dt']

790194    11-04-2018 15:47:52
Name: op_dt, dtype: object

In [81]:
def_list[def_list['Regn No']=='GJ06AX5483']

,S. No,Select Demand Notice,Regn No,Office,Regn Date,Vehicle Class,Vehicle Category,purpose Code,Tax Type,Owner Name,...,Tax Arrear From,Tax Arrear UpTo,Permit Valid From,Permit Valid Upto,Tax Amount in (Rs),Tax Fine in (Rs),Total Tax Amount in (Rs),Fitness Validity,Financer Name,Blacklist Status
1869,1870,False,GJ06AX5483,"JAGATPURA, JAIPUR ARTO(RJ-141)",31-Mar-2017,Goods Carrier,HGV,58,MV Tax,SUN FLAMES PETRO,...,01-Apr-2019,31-Mar-2021,NaN,NaN,"53,069","5,552","58,621",30-Mar-2021,HDFC BANK LTD,No
25221,25222,False,GJ06AX5483,"JAGATPURA, JAIPUR ARTO(RJ-141)",31-Mar-2017,Goods Carrier,HGV,97,Special Tax,SUN FLAMES PETRO,...,01-Apr-2019,31-Mar-2021,NaN,NaN,"13,984","1,574","15,558",30-Mar-2021,HDFC BANK LTD,No


In [85]:
vt_tax[vt_tax['Regn No']=='GJ06AX5483']

,Regn No,Tax Mode,Payment Mode,Tax Amt,Tax Fine,Rcpt No,Rcpt Dt,Tax From,Tax Upto,Pur Cd,Flag,Collected By,State Cd,Off Cd
7699575,GJ06AX5483,Y,M,13527,399,RJ141R1804003371,2018-04-11 15:47:51.647,2018-03-26,2019-03-31,58,NaN,1612007479,RJ,141


In [371]:
vt_tax_based_on.columns

Index(['Number of Records', 'ac_fitted', 'audio_fitted', 'cubic_cap',
       'distance_run_in_quarter', 'domain_cd', 'fin_yr_sale_amt', 'floor_area',
       'fuel', 'gcw', 'height', 'hp', 'imported_vch', 'ld_wt', 'length',
       'no_adv_units', 'no_cyl', 'no_of_trips', 'off_cd', 'op_dt',
       'other_criteria', 'pmt_catg', 'pmt_type', 'purchase_dt', 'rcpt_no',
       'regn_no', 'regn_type', 'route_class', 'route_length', 'sale_amt',
       'seat_cap', 'service_type', 'sleeper_cap', 'stand_cap', 'state_cd',
       'unld_wt', 'vch_catg', 'vch_purchase_as', 'vh_class', 'video_fitted',
       'wheelbase', 'width'],
      dtype='object')